# MM High Risk - Train Model
Multiple Myeloma (MM) is a type of bone marrow cancer. Treatment for MM involves combinations of drugs over multiple cycles. There is huge heterogeneity in treatment response with some individuals not responding to treatment and some patients responding well to treatment for some time before a relapse. A better characterization of patients who relapse early can influence the treatment options and combinations.

## Objective
Develop a machine learning model for predicting the risk of fast dying or relapsing of newly diagnosed MM patients.

## Import libraries

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

from pathlib import Path

In [2]:
# autoreload changes from local files
%load_ext autoreload
%autoreload 2

# pandas show full output
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

# add module path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src import config
from src import preprocess
from src import model
from src import visual
from src import gene

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

## Load data

Load training and testing data. In case you haven't created the training and testing data yet in the previous notebook, then uncomment and run the line below.

In [5]:
# df_model = preprocess.create_model_input_data(save=True)

In [6]:
df_train = preprocess.load_training_data()
df_test = preprocess.load_testing_data()
df_train.head()

,D_Age,D_Gender,D_ISS,CYTO_predicted_feature_01,CYTO_predicted_feature_02,CYTO_predicted_feature_03,CYTO_predicted_feature_05,CYTO_predicted_feature_06,CYTO_predicted_feature_08,CYTO_predicted_feature_10,CYTO_predicted_feature_12,CYTO_predicted_feature_13,CYTO_predicted_feature_14,CYTO_predicted_feature_15,CYTO_predicted_feature_16,CYTO_predicted_feature_17,CYTO_predicted_feature_18,HR_FLAG,Entrez_1,Entrez_2,Entrez_3,Entrez_9,Entrez_10,Entrez_13,Entrez_14,Entrez_15,Entrez_16,Entrez_18,Entrez_19,Entrez_20,Entrez_21,Entrez_22,Entrez_23,Entrez_24,Entrez_25,Entrez_26,Entrez_27,Entrez_28,Entrez_29,Entrez_30,Entrez_32,Entrez_33,Entrez_34,Entrez_35,Entrez_36,Entrez_37,Entrez_38,Entrez_39,Entrez_40,Entrez_41,Entrez_43,Entrez_47,Entrez_48,Entrez_49,Entrez_50,Entrez_51,Entrez_52,Entrez_53,Entrez_54,Entrez_55,Entrez_56,Entrez_58,Entrez_59,Entrez_60,Entrez_70,Entrez_71,Entrez_72,Entrez_81,Entrez_86,Entrez_87,Entrez_88,Entrez_89,Entrez_90,Entrez_91,Entrez_92,Entrez_93,Entrez_94,Entrez_95,Entrez_97,Entrez_98,Entrez_100,Entrez_101,Entrez_102,Entrez_103,Entrez_104,Entrez_105,Entrez_107,Entrez_108,Entrez_109,Entrez_111,Entrez_112,Entrez_113,Entrez_114,Entrez_115,Entrez_116,Entrez_117,Entrez_118,Entrez_119,Entrez_120,Entrez_123,...,Entrez_252953,Entrez_252955,Entrez_2576,Entrez_26240,Entrez_26267,Entrez_2657,Entrez_26628,Entrez_27183,Entrez_27328,Entrez_284194,Entrez_284366,Entrez_2844,Entrez_286128,Entrez_29940,Entrez_29994,Entrez_3117,Entrez_3222,Entrez_3316,Entrez_344,Entrez_3690,Entrez_3742,Entrez_3753,Entrez_378108,Entrez_378948,Entrez_387104,Entrez_388389,Entrez_3963,Entrez_3987,Entrez_401428,Entrez_414245,Entrez_4253,Entrez_440574,Entrez_440895,Entrez_4701,Entrez_4714,Entrez_50858,Entrez_5098,Entrez_51124,Entrez_51263,Entrez_51326,Entrez_51643,Entrez_51735,Entrez_51750,Entrez_5296,Entrez_53916,Entrez_54949,Entrez_552900,Entrez_55308,Entrez_5683,Entrez_572558,Entrez_57335,Entrez_57497,Entrez_57501,Entrez_582,Entrez_58496,Entrez_5940,Entrez_60677,Entrez_6139,Entrez_641367,Entrez_641517,Entrez_642778,Entrez_645166,Entrez_64788,Entrez_65082,Entrez_653067,Entrez_65988,Entrez_6606,Entrez_6844,Entrez_727856,Entrez_728047,Entrez_728411,Entrez_728734,Entrez_731275,Entrez_7730,Entrez_79086,Entrez_79741,Entrez_80006,Entrez_805,Entrez_80829,Entrez_8190,Entrez_8293,Entrez_8302,Entrez_83864,Entrez_83986,Entrez_84220,Entrez_84342,Entrez_84619,Entrez_84672,Entrez_84673,Entrez_8490,Entrez_84976,Entrez_8509,Entrez_86614,Entrez_8778,Entrez_8926,Entrez_9570,Entrez_9692,Entrez_9720,Entrez_9768,Entrez_9797
0,53,Female,2.0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,1,0,5.537100,0.319986,0.0,1.326200,0.0,0.0,32.2357,1.077890,40.0047,0.370424,0.456427,0.832329,1.696350,4.58103,20.0967,0.002118,7.30377,0.038114,2.63084,0.019251,1.129040,17.5644,0.103975,0.000000,19.1865,0.176863,4.59999,241.2090,41.4961,5.35351,0.000000,0.000000,15.502500,22.1528,4.44696,0.011813,26.8060,8.71236,32.0205,9.20201,24.25310,0.037352,0.007352,0.169543,0.728360,872.290,0.000000,296.965,0.038783,39.9283,16.1697,0.536849,0.000000,0.000000,4.44569,1.62064,3.27488,1.91056,0.035025,11.51650,2.73428,15.02490,9.09920,14.76040,17.0488,109.2530,2.47428,0.000000,0.019648,0.000000,3.229060,0.026659,1.823080,15.26810,0.0,0.052030,0.000000,0.0,51.8207,0.255654,22.20760,6.583470,...,0.0,0.0,0.00000,0.558980,2.440529,0.025282,0.124154,16.634899,0.004964,0.461558,0.0,3.730470,1.410060,1.352189,4.215457,0.151257,0.0,0.010208,0.000000,0.845080,0.003399,0.046065,1.209968,0.0,0.000000,0.039299,0.000000,5.542835,2.315734,0.829441,7.42140,17.527550,0.046833,14.34540,32.917805,0.0,0.992782,11.871750,1.985060,3.223010,35.125510,1.623395,2.749050,4.048090,7.501123,12.981500,32.062850,5.159106,36.179935,0.0,0.808360,0.0,0.033320,2.966372,2.070388,0.0,0.131731,342.957695,0.011962,0.0,16.112854,1.529000,15.672136,2.158727,0.033352,1.300748,6.105335,24.937650,0.087259,0.822156,2.313015,0.006812,2.431810,0.379061,16.58383,0.865157,3.229735,74.69845,8.957061,0.024767,4.435979,0.000000,0.000000,4.214925,11.913600,3.979330,7.805535,

Split X and y columns.

In [7]:
X_train, y_train = preprocess.split_x_y(df_train)
X_test, y_test = preprocess.split_x_y(df_test)

## Train model

Models are selected based on data characteristics. In this particular case, the classifier needs to perform well on small datasets with large dimensionality. Ensemble methods have shown good performance for these types of datasets, as these train multiple models over various partitions in the feature space. An additional benefit is that these models are robust against outliers and multicollinearity, which helps with the large set of gene expression data for which the actual possible value ranges are unknown.

Therefore, subsequent experiments use two types of classifiers: RandomForest (RF) and a XGBoost  (XGB). 

For both classifiers three pipelines are developed and evaluated – totalling in 6 different pipeline combinations. One of the main challenges that needs to be addressed is the large input feature space. Hence, each pipeline applies a strategy to reduce the feature set and thus minimize the complexity of the model, which in turn will likely improve the performance of the model.

The following pipelines are implemented. For visualizations of the pipelines, see the Appendix.
- Baseline: results of the DREAM Challenge article show that a classifier with only four features (i.e. Age, ISS, PHF19, and MMSET) performs virtually identical to more complex models. The article, however, does not specify which classifier and pipeline is used.
- Feature selection: apply a set of three feature selection techniques – 1) remove constant features, 2) perform a quick univariate test to obtain 1000 best features, 3) reduce space even further with a slower but more sophisticated recursive feature elimination (RFECV). 
- Dimensionality reduction: apply principal component analysis (PCA) to the gene expression columns to obtain vectors that explain most of the variance in the dataset.

### Baseline pipeline

In [63]:
model_names = ['RF_baseline', 'XGB_baseline']

X_train_min = X_train[config.FEATURES_MINIMAL].copy()
X_test_min = X_test[config.FEATURES_MINIMAL].copy()

# todo: look for pipeline winners

pipelines = [
    model.add_clf_rf(model.get_pipeline_transformers_baseline(X_train_min)),
    model.add_clf_xgb(model.get_pipeline_transformers_baseline(X_train_min))
]
param_grids = [
    model.get_param_grid_rf(prefix='clf__'),                
    model.get_param_grid_xgb(prefix='clf__')
]


In [64]:
clfs_base = model.train_multiple_models(X_train_min, y_train, X_test_min, y_test, model_names, pipelines, param_grids, n_jobs=4, cv=3, n_iter=10, verbose=10)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model saved to: C:\projects\side_projects\mm_highrisk\models\model_RF_baseline_2022-11-13.joblib
RF_baseline - average_precision test data: 0.4781543692907255 

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model saved to: C:\projects\side_projects\mm_highrisk\models\model_XGB_baseline_2022-11-13.joblib
XGB_baseline - average_precision test data: 0.4291281014526167 



### Feature selection pipeline
Since the input data is quite large in terms of features, we need to reduce the feature space significantly. Hence, this implementation is a pipeline that has a three feature selection steps:
- VarianceThreshold: remove features with constant value (non-informative)
- SelectKBest: perform ANOVA test to quickly reduce the feature space
- RFECV: recursively eliminate features that have the least impact on model performance. Computationally expensive.

By performing the first two selection steps, we reduce the feature space considerably. Afterwards a more sophisticated technique like RFECV can be applied to reduce the space even further. This way we minimize the computation time, while still using advanced feature selection techniques.

In [61]:
model_names = ['RF_select', 'XGB_select']
select_k = 1000

pipelines = [
    model.add_clf_rf(model.get_pipeline_transformers_select(X_train, select_k=select_k)),
    model.add_clf_xgb(model.get_pipeline_transformers_select(X_train, select_k=select_k))
]
param_grids = [
    model.get_param_grid_rf(prefix='clf__'),                
    model.get_param_grid_xgb(prefix='clf__')
]

In [62]:
clfs_select = model.train_multiple_models(X_train, y_train, X_test, y_test, model_names, pipelines, param_grids, n_jobs=4, cv=3, n_iter=10, verbose=10)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\dkn1904.51564\Miniconda3\envs\mm_highrisk\lib\site-packages\sklearn\pipeline.py:336: UserWarning: Persisting input arguments took 0.82s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


Model saved to: C:\projects\side_projects\mm_highrisk\models\model_RF_select_2022-11-13.joblib
RF_select - average_precision test data: 0.5368288067509762 

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\dkn1904.51564\Miniconda3\envs\mm_highrisk\lib\site-packages\sklearn\pipeline.py:336: UserWarning: Persisting input arguments took 0.72s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


Model saved to: C:\projects\side_projects\mm_highrisk\models\model_XGB_select_2022-11-13.joblib
XGB_select - average_precision test data: 0.47815009234166383 



## Dimensionality reduction pipeline
Another approach to reduce the feature space is by applying dimensionality reduction techniques like PCA. We apply the following pipeline.

Notice that we only perform PCA on the continuous columns, i.e. the gene expression columns.

In [13]:
model_names = ['RF_dimred', 'XGB_dimred']

pipelines = [
    model.add_clf_rf(model.get_pipeline_transformers_pca(X_train)),
    model.add_clf_xgb(model.get_pipeline_transformers_pca(X_train))
]

pca_params = {"preprocess__pca_continuous__pca__n_components": range(5, 200, 5)}
param_grids = [
    pca_params | model.get_param_grid_rf(prefix='clf__'),
    pca_params | model.get_param_grid_xgb(prefix='clf__')
]

In [9]:
clfs_dimred = model.train_multiple_models(X_train, y_train, X_test, y_test, model_names, pipelines, param_grids, n_jobs=4, cv=3, n_iter=10, verbose=10)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model saved to: C:\projects\side_projects\mm_highrisk\models\model_XGB_dimred_2022-11-13.joblib
XGB_dimred - average_precision test data: 0.46012668583531124 



## Conclusion
We trained in total 6 pipelines. In the next notebook will evaluate the performance of these models on the test set.